In [1]:
import pandas as pd
import numpy as np
import spacy
import requests
from bs4 import BeautifulSoup
import re
import pygal
import scattertext as st
import imp
#imp.reload('scattertext')
from IPython.display import IFrame
from IPython.core.display import display, HTML
from collections import Counter
import seaborn as sns
from scipy.stats import rankdata, hmean
display(HTML("<style>.container { width:98% !important; }</style>"))
%matplotlib inline

In [2]:
nlp = spacy.en.English()

In [3]:
convention_df = st.SampleCorpora.ConventionData2012.get_data()
convention_df['parsed'] = convention_df['text'].apply(nlp)

In [4]:
corpus = st.CorpusFromParsedDocuments(convention_df, category_col='party', parsed_col='parsed').build()

In [5]:
import numpy as np
from statsmodels.regression.linear_model import OLS


class OLSUngarStyle(object):
	@staticmethod    
	def get_scores_and_p_values(tdm, category):
		'''
		Parameters
		----------
		tdm: TermDocMatrix
		category: str, category name

		Returns
		-------
		pd.DataFrame(['coef', 'p-val'])
		'''
		X = tdm._X
		y = (tdm.get_category_names_by_row() == category).astype(int) * 2 - 1.
		pX = X/X.sum(axis=1)
		ansX = pX.copy()
		ansX = 2 * np.sqrt(np.array(ansX) + 3./8)
		res = OLS(y, ansX).fit()
		return res


In [15]:
X = corpus._X

In [12]:
category = 'democrat'
y = (corpus.get_category_names_by_row() == category).astype(int) * 2 - 1.

In [16]:
ansX = 2* np.sqrt(X/X.sum(axis=1) + 3./8)

In [18]:
y.shape, ansX.shape

((189,), (189, 62257))

In [9]:
#from statsmodels.regression.linear_model import OLS
#res = OLS(y, 2* np.sqrt(pX + 3./8)).fit()

In [ ]:
from scipy import stats
reg = OLS(y, ansX).fit()

In [ ]:
#ps = reg.pvalues()

In [ ]:
reg.summary()

In [ ]:
reg

#def zero_rank(data):

data = np.array([5, -2, 3, 10, 16, 1, -1])
neg_rank = scaled_rank(data[data < 0])*0.5
pos_ranks = scaled_rank(data[data >= 0])*0.5 + 0.5


In [15]:
def scaled_rank(data):
    return scaled_frequency(rankdata(data, 'ordinal'))

def rank_zero_at_half(data):
    data_out = np.copy(data)
    data_out[data < 0] = scaled_rank(data[data < 0])*0.5
    data_out[data >= 0] = scaled_rank(data[data >= 0])*0.5 + 0.5
    return data_out

def scaled_frequency(ranks):
    return (ranks.astype(float) - ranks.min())/(ranks.max() - ranks.min())

def scaled_log_freq(ranks):
    ranks = np.log(ranks+1)
    return (ranks.astype(float) - ranks.min())/(ranks.max() - ranks.min())

def find_most_common_pos(parsed_col, corpus, word):
    c = Counter()
    for parsed in corpus.search(word)['parsed']:
        for w in parsed:
            if w.lower_ == word:
                c[w.pos_] += 1
    return c.most_common(1)[0][0]

def get_should_be_there_but_isnt(corpus, 
                                 min_background_corpus_rank= 0.9, 
                                 min_corpus_rank = 0.5, 
                                 max_background_rank = 0.999, 
                                 min_similarity_rank = 0.7,
                                 size_of_distictive_set = 10, 
                                 filter_proper_nouns = True):
    assert type(corpus) == st.ParsedCorpus
    unigram_corpus = corpus.get_unigram_corpus()
    background_freq_df = unigram_corpus.get_scaled_f_scores_vs_background()
    background_freq_df = background_freq_df.loc[unigram_corpus.get_term_freq_df().index].fillna(0)
    background_freq_df['corpus_rank'] = scaled_rank(background_freq_df['corpus'])
    background_freq_df['background_rank'] = scaled_rank(background_freq_df['background'])
    most_distinctive_terms = ([nlp(term) 
                              for term 
                              in (background_freq_df[background_freq_df['background'] > 0]
                                  .sort_values(by='Scaled f-score', ascending=False)
                                  .iloc[:size_of_distictive_set*10]
                                  .index)
                              if not filter_proper_nouns 
                               or find_most_common_pos('parsed', corpus, term) != 'PROPN']
                              [:size_of_distictive_set])
    background_freq_df['semantic_similarity'] = [np.max([nlp(term).similarity(distinctive_term) for distinctive_term in most_distinctive_terms]) 
                                                 for term in background_freq_df.index]
    background_freq_df['semantic_similarity_rank'] = ((rankdata(background_freq_df['semantic_similarity'])+1.)/(len(background_freq_df)+1.))

    new_stop_list = (set(background_freq_df[((background_freq_df['background_rank'] < min_background_corpus_rank)
                                             & (background_freq_df['corpus_rank'] < min_corpus_rank))].index) 
                     | set(t for t in background_freq_df.index if len(t) == 1)
                     | set(background_freq_df[background_freq_df['background'] == 0].index)
                     | set(background_freq_df[background_freq_df['background_rank'] > max_background_rank].index)
                     | set(background_freq_df[background_freq_df['semantic_similarity_rank'] < min_similarity_rank].index)
                    )
    print(len(new_stop_list), len(background_freq_df))
    unigram_corpus = unigram_corpus.remove_terms(new_stop_list)
    #unigram_corpus = corpus.get_unigram_corpus()
    #background_freq_df = background_freq_df.loc[set(background_freq_df.index) - new_stop_list]
    background_freq_df = unigram_corpus.get_scaled_f_scores_vs_background()
    background_freq_df = background_freq_df.loc[unigram_corpus.get_term_freq_df().index].fillna(0)
    background_freq_df['corpus_rank'] = scaled_rank(background_freq_df['corpus'])
    background_freq_df['background_rank'] = scaled_rank(background_freq_df['background'])
    background_freq_df['semantic_similarity'] = [np.max([nlp(term).similarity(distinctive_term) 
                                                         for distinctive_term in most_distinctive_terms])
                                                 for term in background_freq_df.index]
    #background_freq_df = background_freq_df.sort_values(by='corpus_background_delta_pctl', ascending=False)
    background_freq_df['background_delta'] = background_freq_df['background_rank'] - background_freq_df['corpus_rank']
    background_freq_df['background_delta_rank'] = rank_zero_at_half(background_freq_df['background_rank'] - background_freq_df['corpus_rank'])
    background_freq_df['semantic_similarity_rank'] = ((rankdata(background_freq_df['semantic_similarity'])+1.)/(len(background_freq_df)+1.))

    #background_freq_df['should_be_there'] = hmean([background_freq_df['background_delta_rank'], background_freq_df['semantic_similarity_rank']])
    background_freq_df['should_be_there'] = hmean([background_freq_df['background_delta_rank']+0.0001, background_freq_df['semantic_similarity_rank']+0.0001])

    print(len(background_freq_df))
    return unigram_corpus, background_freq_df, most_distinctive_terms

In [16]:
unigram_corpus, background_freq_df, most_distinctive_terms = get_should_be_there_but_isnt(corpus)
print(most_distinctive_terms)

6658 8160
1502
[hardworking, bailouts, autoworkers, grandkids, billionaires, millionaires, noches, bless, dreamers, outcompete]


In [17]:
#background_freq_df.sort_values(by='semantic_similarity', ascending=False) 
from scipy.stats import hmean, rankdata

#corpus_background_delta_pctl = 1-(np.argsort(background_freq_df['background_rank'] - background_freq_df['corpus_rank'])+1.)/(len(background_freq_df)+1)
#background_freq_df['should_be_there_score'] = hmean([semantic_sim_pctl, corpus_background_delta_pctl])
#background_freq_df['corpus_background_delta_pctl'] = corpus_background_delta_pctl
len(background_freq_df)

1502

In [18]:
html = st.produce_scattertext_explorer(unigram_corpus,
                                       category='democrat',
                                       category_name='Characteristic',
                                       not_category_name='Not Characteristic',
                                       minimum_term_frequency=0,
                                       width_in_pixels=1200,
                                       x_coords=background_freq_df['background_delta_rank'],
                                       y_coords=background_freq_df['semantic_similarity_rank'],
                                       scores=1.-background_freq_df['should_be_there'],
                                       sort_by_dist=False,
                                       show_characteristic=False,
                                       metadata=convention_df['speaker'],
                                       x_label='Background Delta',
                                       y_label='Similarity to Characteristic Terms')
fn = 'demo_expected_vs_actual.html'
open(fn, 'wb').write(html.encode('utf-8'))
IFrame(src=fn, width = 1400, height=1000)

In [19]:
debate_df = pd.read_csv('../scattertext/scattertext/data/presidential_debates_2016.csv.gz')
debate_df = debate_df[debate_df['party'].isin(['Democratic', 'Republican'])]
debate_df['parsed'] = debate_df['statement'].apply(nlp)
debate_corpus = st.CorpusFromParsedDocuments(debate_df, category_col='party', parsed_col='parsed').build()

In [20]:
debate_unigram_corpus, debate_background_freq_df, distinctive_terms = get_should_be_there_but_isnt(debate_corpus, filter_proper_nouns=False)
print(distinctive_terms)

3096 3748
652
[obama, outsmarted, barack, twitter, trump, kaine, hillary, frisk, renegotiate, slobs]


In [21]:
html = st.produce_scattertext_explorer(debate_unigram_corpus,
                                       category='Democratic',
                                       category_name='Characteristic',
                                       not_category_name='Not Characteristic',
                                       minimum_term_frequency=0,
                                       width_in_pixels=1200,
                                       x_coords=debate_background_freq_df['background_delta_rank'],
                                       y_coords=debate_background_freq_df['semantic_similarity_rank'],
                                       scores=1.-debate_background_freq_df['should_be_there'],
                                       sort_by_dist=False,
                                       show_characteristic=False,
                                       metadata=debate_df['speaker and debate'],
                                       x_label='Background Delta',
                                       y_label='Similarity to Characteristic Terms')
fn = 'demo_expected_vs_actual.html'
open(fn, 'wb').write(html.encode('utf-8'))
IFrame(src=fn, width = 1400, height=1000)